In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def electricity_mix_region(file_path, run_name,output_folder, called = False):
    if called == True:

        values1 = np.linspace(0, 1, 20)
        values2 = np.linspace(0, 1, 20)
        values3 = np.linspace(0, 1, 16)
        # Create two colormaps
        cmap1 = plt.cm.tab20
        cmap2 = plt.cm.tab20b
        cmap3 = plt.cm.Set3
        colors1 = cmap1(values1)
        colors2 = cmap2(values2)
        colors3 = cmap3(values3)

        combined_colors = np.vstack((colors1, colors2,colors3))
        combined_cmap = ListedColormap(combined_colors)

        var_Fout = pd.read_csv(file_path + 'VAR_FOut_' + run_name + '.csv', sep = ',')
        elec_prod = var_Fout[var_Fout['1'] == 'ELC']
        unique_countries = elec_prod['4'].unique()
        list_unique_countries = [unique_countries[0:20],unique_countries[20:]]
        c = 0
        for list_countries in list_unique_countries:
            c+=1
            unique_countries1 = list_countries
            # Calculate the number of rows and columns for subplots
            num_countries = len(unique_countries1)
            num_cols = 4  # Number of columns
            num_rows = -(-num_countries // num_cols)  # Ceiling division to calculate number of rows

            fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 4 * num_rows))

            # Flatten the axes array if only one row of subplots
            if num_rows == 1:
                axes = axes.ravel()
            else:
                axes = [item for sublist in axes for item in sublist]

            for ax, country in zip(axes, unique_countries1):
                
                elec_prod = var_Fout[var_Fout['1'] == 'ELC']

                elec_prod['prod_type'] = elec_prod['2'].str.split('_').str[1]
                elec_prod['group_type'] = elec_prod['prod_type'].astype(str).str.cat(elec_prod['3'].astype(str), sep='_')
                elec_prod['group_prod_type'] = elec_prod.groupby('group_type')['8'].transform('sum')
                elec_prod = elec_prod[~elec_prod['group_type'].duplicated()]

                pivot_df_elec = elec_prod.pivot(index='3', columns='prod_type', values='group_prod_type')
                column_sums_elec = pivot_df_elec.sum(skipna=True)

                # Step 2: Sort the columns based on their sums
                sorted_columns_elec = column_sums_elec.sort_values(ascending=False)

                # Step 3: Reindex the DataFrame with the sorted column order
                sorted_df_elec = pivot_df_elec.reindex(sorted_columns_elec.index, axis=1)

                elec_prod = var_Fout[var_Fout['1'] == 'ELC']
                subset_df = elec_prod[elec_prod['4'] == country]
                
                subset_df['prod_type'] = subset_df['2'].str.split('_').str[1]
                subset_df['group_type'] = subset_df['prod_type'].astype(str).str.cat(subset_df['3'].astype(str), sep='_')
                subset_df['group_prod_type'] = subset_df.groupby('group_type')['8'].transform('sum')
                subset_df = subset_df[~subset_df['group_type'].duplicated()]
                pivot_df = subset_df.pivot(index='3', columns='prod_type', values='group_prod_type')

                # Step 3: Reindex the DataFrame with the sorted column order
                sorted_df = pivot_df.reindex(sorted_columns_elec.index, axis=1)
                
                # Plot with the discrete color map
                sorted_df.plot(kind='bar',ax=ax, stacked=True, colormap=combined_cmap,legend=False)
                
                
                # Set labels and title
                ax.set_xlabel('Year')
                ax.set_ylabel('GWh')
                ax.set_title(f'Region {country}')
                # ax.set_xticks(subset_df['Month'])
                # ax.set_xticklabels(subset_df['Month'], rotation=45)
                
                # Show legend only on the last subplo
                
                ax.grid(False)

            handles, labels = ax.get_legend_handles_labels()
            fig.legend(handles, labels, bbox_to_anchor=(0.5, -0.08), loc = 'lower center', ncol=10)
            # Adjust layout
            plt.tight_layout()
            plt.savefig(output_folder + f'elec_mix_region{c}.pdf'
                        , format ='pdf'
                        ,  bbox_inches='tight')
    else:
        pass